In [2]:
import numpy
import torch
from torch.utils import data

In [3]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w)+b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [6]:
t_w = torch.tensor([2, -3.4])
t_b = 4.2
features, labels = synthetic_data(t_w, t_b, 1000)

In [7]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)    

优化的地方主要是DataLoader，可以自动shuffle并且可以自动给出迭代批次

In [8]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)

In [9]:
next(iter(data_iter))

[tensor([[-0.9879,  0.9485],
         [ 0.6505, -0.2111],
         [-0.3458, -0.5826],
         [-0.8377, -0.4560],
         [ 0.2452,  0.6834],
         [-1.2313, -1.2496],
         [ 0.8437, -0.0974],
         [ 0.5345, -2.1502],
         [-0.9392,  0.4653],
         [ 1.6840, -1.2979]]),
 tensor([[-1.0122],
         [ 6.2299],
         [ 5.4853],
         [ 4.0815],
         [ 2.3572],
         [ 5.9898],
         [ 6.2325],
         [12.5799],
         [ 0.7421],
         [11.9765]])]

In [14]:
from torch import nn
net = nn.Sequential(nn.Linear(2, 1))
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)
loss=nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

loss简化，网络简化，直接用Linear就行，参数也简化， 优化函数也包装好了

In [15]:
num_epoch = 3
for epoch in range(num_epoch):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch+1}, loss {l:f}')

epoch 1, loss 0.000171
epoch 2, loss 0.000100
epoch 3, loss 0.000100


In [16]:
w = net[0].weight.data
print(t_w-w.reshape(t_w.shape))
b=net[0].bias.data
print(t_b-b)

tensor([ 0.0004, -0.0004])
tensor([-0.0002])
